# Принцип работы 
Текстовое описание видео, полученное из полей "fulltitle", "description" мы: 

*  переведём предложение в нормальный вид, если оно набрано в верхнем регистре;
*  очистим от текст от специальных и обычных символов, двойных пробелов, ограничение по минимальной длинне для векторизации текста;
*  лемматизацию не делаем, т.к. работаем с векторами BERT
*  разобъём текст на предложения;
*  предложения разобъём на ngram, длинной 2 - 6 слов;
*  если в в ngram на первом или последнем месте стоит слово из топа 500 по закону распределению Ципфа, то такие ngram игнорируем;
*  получим вектора для предложений и ngram;
*  извлечём дополнительные признаки: 
    *  признак извлечения текста из предложения или определённой ngram;
    *  продолжительность (по логарифмической шкале, основание логарифма = 10);
    *  доля лайков по отношению к количеству просмотра;
    *  доля дизлайков по отношению к количеству просмотров;
    *  доля комментариев по отношению к количеству просмотров;
    *  дата загрузки;






# ИМПОРТ



In [ ]:
# A dependency of the preprocessing for BERT inputs
!pip install -q -U tensorflow-text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 47.0 MB/s eta 0:00:00


In [ ]:
import os
import sys
import re
import time
import math
import pandas as pd
import numpy as np
import pprint
import tempfile
from typing import Dict, Text

from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import cosine 
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
from nltk.util import ngrams

import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.layers import Embedding, Flatten, Input, concatenate, Reshape
from tensorflow.keras.utils import plot_model, to_categorical
from tensorflow.keras.optimizers import Adam, Adadelta
from tensorflow.keras.callbacks import ModelCheckpoint, Callback
import tensorflow_hub as hub
import tensorflow_text as text

from matplotlib import pyplot as plt

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
PATH = '/content/drive/MyDrive/Colab Notebooks/HACKATHONS/Видеопереворот/Рекомендательная система/files/'

# Обезличеная база данных видеоконтента 
concurs_data = 'concurs_data.csv'

# ДАННЫЕ

## 500 самых частых слов по закону Ципфа

In [ ]:
# 500 Самых частых слов по закону Ципфа
cipf_500 = [
  'и',
  'в',
  'не',
  'на',
  'я',
  'быть',
  'он',
  'с',
  'что',
  'а',
  'по',
  'это',
  'она',
  'этот',
  'к',
  'но',
  'они',
  'мы',
  'как',
  'из',
  'у',
  'который',
  'то',
  'за',
  'свой',
  'что',
  'весь',
  'год',
  'от',
  'так',
  'о',
  'для',
  'ты',
  'же',
  'все',
  'тот',
  'мочь',
  'вы',
  'человек',
  'такой',
  'его',
  'сказать',
  'только',
  'или',
  'ещё',
  'бы',
  'себя',
  'один',
  'как',
  'уже',
  'до',
  'время',
  'если',
  'сам',
  'когда',
  'другой',
  'вот',
  'говорить',
  'наш',
  'мой',
  'знать',
  'стать',
  'при',
  'чтобы',
  'дело',
  'жизнь',
  'кто',
  'первый',
  'очень',
  'два',
  'день',
  'её',
  'новый',
  'рука',
  'даже',
  'во',
  'со',
  'раз',
  'где',
  'там',
  'под',
  'можно',
  'ну',
  'какой',
  'после',
  'их',
  'работа',
  'без',
  'самый',
  'потом',
  'надо',
  'хотеть',
  'ли',
  'слово',
  'идти',
  'большой',
  'должен',
  'место',
  'иметь',
  'ничто',
  'то',
  'сейчас',
  'тут',
  'лицо',
  'каждый',
  'друг',
  'нет',
  'теперь',
  'ни',
  'глаз',
  'тоже',
  'тогда',
  'видеть',
  'вопрос',
  'через',
  'да',
  'здесь',
  'дом',
  'да',
  'потому',
  'сторона',
  'какой-то',
  'думать',
  'сделать',
  'страна',
  'жить',
  'чем',
  'мир',
  'об',
  'последний',
  'случай',
  'голова',
  'более',
  'делать',
  'что-то',
  'смотреть',
  'ребенок',
  'просто',
  'конечно',
  'сила',
  'российский',
  'конец',
  'перед',
  'несколько',
  'вид',
  'система',
  'всегда',
  'работать',
  'между',
  'три',
  'нет',
  'понять',
  'пойти',
  'часть',
  'спросить',
  'город',
  'дать',
  'также',
  'никто',
  'понимать',
  'получить',
  'отношение',
  'лишь',
  'второй',
  'именно',
  'ваш',
  'хотя',
  'ни',
  'сидеть',
  'над',
  'женщина',
  'оказаться',
  'русский',
  'один',
  'взять',
  'прийти',
  'являться',
  'деньги',
  'почему',
  'вдруг',
  'любить',
  'стоить',
  'почти',
  'земля',
  'общий',
  'ведь',
  'машина',
  'однако',
  'сразу',
  'хорошо',
  'вода',
  'отец',
  'высокий',
  'остаться',
  'выйти',
  'много',
  'проблема',
  'начать',
  'хороший',
  'час',
  'это',
  'сегодня',
  'право',
  'совсем',
  'нога',
  'считать',
  'главный',
  'решение',
  'увидеть',
  'дверь',
  'казаться',
  'образ',
  'писать',
  'история',
  'лучший',
  'власть',
  'закон',
  'все',
  'война',
  'бог',
  'голос',
  'найти',
  'поэтому',
  'стоять',
  'вообще',
  'тысяча',
  'больше',
  'вместе',
  'маленький',
  'книга',
  'некоторый',
  'решить',
  'любой',
  'возможность',
  'результат',
  'ночь',
  'стол',
  'никогда',
  'имя',
  'область',
  'молодой',
  'пройти',
  'например',
  'статья',
  'оно',
  'число',
  'компания',
  'про',
  'государственный',
  'полный',
  'принять',
  'народ',
  'никакой',
  'советский',
  'жена',
  'настоящий',
  'всякий',
  'группа',
  'развитие',
  'процесс',
  'суд',
  'давать',
  'ответить',
  'старый',
  'условие',
  'твой',
  'пока',
  'средство',
  'помнить',
  'начало',
  'ждать',
  'свет',
  'пора',
  'путь',
  'душа',
  'куда',
  'нужно',
  'разный',
  'нужный',
  'уровень',
  'иной',
  'форма',
  'связь',
  'уж',
  'минута',
  'кроме',
  'находиться',
  'опять',
  'многий',
  'белый',
  'собственный',
  'улица',
  'черный',
  'написать',
  'вечер',
  'снова',
  'основной',
  'качество',
  'мысль',
  'дорога',
  'мать',
  'действие',
  'месяц',
  'оставаться',
  'государство',
  'язык',
  'любовь',
  'взгляд',
  'мама',
  'играть',
  'далекий',
  'лежать',
  'нельзя',
  'век',
  'школа',
  'подумать',
  'уйти',
  'цель',
  'среди',
  'общество',
  'посмотреть',
  'деятельность',
  'организация',
  'кто-то',
  'вернуться',
  'президент',
  'комната',
  'порядок',
  'момент',
  'театр',
  'следовать',
  'читать',
  'письмо',
  'подобный',
  'следующий',
  'утро',
  'особенно',
  'помощь',
  'ситуация',
  'роль',
  'бывать',
  'ходить',
  'рубль',
  'начинать',
  'появиться',
  'смысл',
  'состояние',
  'называть',
  'рядом',
  'квартира',
  'назад',
  'равный',
  'из-за',
  'орган',
  'внимание',
  'тело',
  'труд',
  'прийтись',
  'хотеться',
  'сын',
  'мера',
  'пять',
  'смерть',
  'живой',
  'рынок',
  'программа',
  'задача',
  'предприятие',
  'известный',
  'окно',
  'вести',
  'совершенно',
  'военный',
  'разговор',
  'показать',
  'правительство',
  'важный',
  'семья',
  'великий',
  'производство',
  'простой',
  'значит',
  'третий',
  'сколько',
  'огромный',
  'давно',
  'политический',
  'информация',
  'действительно',
  'положение',
  'поставить',
  'бояться',
  'наконец',
  'центр',
  'происходить',
  'ответ',
  'муж',
  'автор',
  'все-таки',
  'стена',
  'существовать',
  'даже',
  'интерес',
  'становиться',
  'федерация',
  'правило',
  'оба',
  'часто',
  'московский',
  'управление',
  'слышать',
  'быстро',
  'смочь',
  'заметить',
  'как-то',
  'мужчина',
  'долго',
  'правда',
  'идея',
  'партия',
  'иногда',
  'использовать',
  'пытаться',
  'готовый',
  'чуть',
  'зачем',
  'представить',
  'чувствовать',
  'создать',
  'совет',
  'счет',
  'сердце',
  'движение',
  'вещь',
  'материал',
  'неделя',
  'чувство',
  'затем',
  'данный',
  'заниматься',
  'продолжать',
  'красный',
  'глава',
  'ко',
  'слушать',
  'наука',
  'узнать',
  'ряд',
  'газета',
  'причина',
  'против',
  'плечо',
  'современный',
  'цена',
  'план',
  'приехать',
  'речь',
  'четыре',
  'отвечать',
  'точка',
  'основа',
  'товарищ',
  'культура',
  'слишком',
  'рассказывать',
  'вполне',
  'далее',
  'рассказать',
  'данные',
  'представлять',
  'мнение',
  'социальный',
  'около',
  'документ',
  'институт',
  'ход',
  'брать',
  'забыть',
  'проект',
  'ранний',
  'встреча',
  'особый',
  'целый',
  'директор',
  'провести',
  'спать',
  'плохой',
  'может',
  'впрочем',
  'сильный',
  'наверное',
  'скорый',
  'ведь',
  'срок',
  'палец',
  'опыт',
  'помочь',
  'больше',
  'приходить',
  'служба'
]

## Датафрейм

In [ ]:
df_concurs = pd.read_csv(PATH + concurs_data)

In [ ]:
df_concurs.shape

(100000, 12)

In [ ]:
df_concurs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 12 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   _key           100000 non-null  int64 
 1   channel_name   99963 non-null   object
 2   comment_count  100000 non-null  int64 
 3   description    95205 non-null   object
 4   dislike_count  100000 non-null  int64 
 5   duration       100000 non-null  int64 
 6   fulltitle      100000 non-null  object
 7   like_count     100000 non-null  int64 
 8   tags           100000 non-null  object
 9   upload_date    100000 non-null  int64 
 10  uploader_url   99998 non-null   object
 11  view_count     100000 non-null  int64 
dtypes: int64(7), object(5)
memory usage: 9.2+ MB


In [ ]:
df_concurs.head()

,_key,channel_name,comment_count,description,dislike_count,duration,fulltitle,like_count,tags,upload_date,uploader_url,view_count
0,21350050,ZO'R TV,135,Официальный сайт: http://www.zortv.uz\nZO'RTV ...,0,1742,Bitta savol bor 1- son (10.07.2018),437,"[""zo'r tv"" 'zor tv' 'зур тв' 'зор тв']",1531170000,http://www.youtube.com/channel/UCisKdSbTsPt16y...,35735
1,71990270,Мир Белогорья,0,ЧП произошло в Краснояружском Староселье. Напо...,0,36,Супружеская пара подорвалась на боеприпасе неу...,23,['белгород' 'белгородская область' 'белгород н...,1669064400,http://www.youtube.com/@mirbelogor,931
2,61570019,Спорт-Экспресс,45,"Российский лыжник рассказал, каково чувствоать...",0,387,"Сергей Устюгов: «На горе может выключить так, ...",644,[],1578085200,http://www.youtube.com/user/sportexpress,30898
3,22003727,Extra News,0,Министр иностранных дел Венгрии Петер Сийярто ...,0,155,Венгрия удостоила Мантурова высшей госнаграды ...,1,['Новости' 'События' 'политика' 'народы'],1625691600,http://www.youtube.com/channel/UCYasXPsHhTVsn5...,63
4,28432077,Твой Дом - Тюрьма,0,Обязательно подпишитесь на наш канал! Воры в з...,0,220,"Коронован ""Дедом Хасаном"", воевал с Монголом. ...",63,['воры в законе 2019' 'криминал' 'киллеры' 'бл...,1560805200,http://www.youtube.com/user/max17111,18455


## Отберём популярные видео 
Не будем обрабатывать все видео - колаб(бесплатная версия) вылетает, не хватает ресурсов. В этом ноутбуке мы продемонстируем концепт, для возьмём количество данных для демонстрации принципа работы, а "боевые" вектора и данные создадим с помощью утилиты set_data.py на сервере, файл доступен в папке проекта на гитхабе в папке "utils".

In [ ]:
df_popular = df_concurs.sort_values(by='view_count', ascending=False)[0:10]

In [ ]:
df_popular['fulltitle'].head()

79153    Маша и Медведь (Masha and The Bear) - Маша плю...
69863    Маша и Медведь - Спи, моя радость, усни! (Сери...
12905    RC Cars MUD OFF Road — Land Rover Defender 90 ...
10902    RC Cars 4x4 Sands Storm Racing and MUD Action ...
69832    Маша и Медведь - Следы невиданных зверей🐾 (Сер...
Name: fulltitle, dtype: object

# BERT

## Загрузка

In [ ]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/4", trainable=True)

## Инициализация слоев BERT
В приведенном ниже коде мы создаем входной слой, используя tf.keras.layers.Input метод. Мы будем использовать в **preprocessed_text** качестве входных данных для этого слоя.

Затем **bert_encoder** функция преобразует предварительно обработанный текст в embadding векторы. Это будет выход этого слоя. Затем **outputs** они будут переданы в слои нейронной сети.

In [ ]:
text_input = Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

## Проверим работу

In [ ]:
p_text = bert_preprocess(['text'])
v = bert_encoder(p_text)

In [ ]:
v['pooled_output'][0]

<tf.Tensor: shape=(768,), dtype=float32, numpy=
array([ 2.70360082e-01, -1.51426017e-01,  2.02479690e-01,  1.36259735e-01,
       -1.11354217e-01,  2.72155672e-01,  1.81260586e-01,  2.49676630e-01,
       -4.55122530e-01,  2.98245847e-01,  2.79911589e-02, -6.94658905e-02,
       -1.89139619e-01, -1.81658924e-01,  3.23850587e-02, -1.17484927e-01,
        4.35243487e-01,  8.09016544e-03, -1.77940235e-01, -4.19278741e-01,
       -9.99817014e-01, -2.44627044e-01, -3.81946921e-01, -2.06391916e-01,
       -3.17560732e-01,  1.76368654e-01, -6.06947541e-02,  2.72821367e-01,
        3.63530397e-01, -1.30347505e-01,  3.64892185e-04, -9.99835253e-01,
        4.42686290e-01,  5.91138065e-01,  2.33706027e-01, -8.11598971e-02,
        1.85111642e-01,  1.85093209e-01,  3.22861195e-01, -3.63657027e-01,
        2.37809997e-02, -1.49478361e-01,  1.35586590e-01,  1.46379545e-01,
        1.47330984e-01, -3.42431307e-01, -8.86009410e-02,  6.14471436e-02,
       -3.10208708e-01,  5.89239858e-02, -1.73562448

# ФУНКЦИИ

In [ ]:
# Очищаем текст регулярными выражениями
def clearText(text):
  text = re.sub(r'[\n\t]', '', text).strip()
  text = re.sub(r'[^\w\d\s]', '', text)
  text = re.sub(r'\s+', ' ', text)
  if len(text) < 3:
    return False
  return text


# Получение спска ngramm
def getNgram(n, sent):
  ngram_list = []
  word_list = sent.split()
  ngrm = ngrams(word_list, n)
  # Перебираем полученные n-граммы
  for ng in ngrm:
    # Если первое или последнее слово в списке cipf_500 - пропускаем n-грамму
    if ng[0] in cipf_500 or ng[-1] in cipf_500:
      continue

    ng_str = ' '.join(ng).strip()
    # Если в n-грамме нет слова (от 3х букв) - пропускаем n-грамму
    if re.search(r'[^a-zA-zа-яёА-ЯЁ]{3,}', ng_str):
      continue

    # print('NG', n, ng_str)
    ngram_list.append(ng_str)

  return ngram_list


# Получаем данные и свойства из строки датафремма
def getRowData(row, vector_type):
  key = row['_key']
  fulltitle = row['fulltitle']
  duration = round(math.log10(row['duration']), 4)
  like_proportion = round(row['like_count'] / row['view_count'], 6)
  dislike_proportion = round(row['dislike_count'] / row['view_count'], 6)
  comment_proportion = round(row['comment_count'] / row['view_count'], 6)	
  data_list = [key, fulltitle, vector_type, duration, like_proportion, dislike_proportion, comment_proportion, row['upload_date']]
  return data_list

# СОЗДАДИМ BERT ВЕКТОРА И ДАННЫЕ С ПРИЗНАКАМИ
В рабочей версии - вектор должен создаваться при загрузке видео, а не на этом шаге - для того, что бы сократить количество операций.

In [ ]:
time_start = time.time()
data_list = []
vector_list = []

for i in range(df_popular.shape[0]):
  df_row = df_popular.iloc[i]

  if pd.notna(df_row['fulltitle']):
    # Разбираем на предложения
    fulltitle_arr = sent_tokenize(df_row['fulltitle'])
    # Проходим по каждому предложению
  
    for sent in fulltitle_arr:
      sent = clearText(sent)
      # Если предложение короче установленной длинны (3 символа)- пропускаем
      if not sent:
        continue
      # Если все буквы в предложении в верхнем регистре - приводим предложение в нормальный вид
      check_letter = False

      for letter in sent:
        if letter.islower():
          check_letter = True
          continue

      if not check_letter:
        # Приводим предложение к нормальному виду
        sent = sent.lower() 
        sent = sent.capitalize()

      # Получаем вектор предложения
      vector = bert_encoder(bert_preprocess([sent]))['pooled_output'][0]
      vector_list.append(vector)

      # Получаем данные и свойства (тип вектора = 7, предложение)
      data = getRowData(df_row, 7)
      data_list.append(data)

      # N-граммы
      n_list = [2, 3, 4, 5, 6]
      ngram_list = []
      for n in n_list:
        ng_list = getNgram(n, sent)
        # Перебираем список n-грамм
        for ng in ng_list:
          # Получаем вектор n-граммы
          vector = bert_encoder(bert_preprocess([ng]))['pooled_output'][0]
          vector_list.append(vector)
          # Получаем данные и свойства (тип вектора = 7, предложение)
          data = getRowData(df_row, n)
          data_list.append(data)

# ПОСМОТРИМ ДАННЫЕ
Данне для демонстрации принципа работы, а "боевые" вектора и данные создадим с помощью утилиты set_data.py на сервере.

In [ ]:
vector_list[0]

<tf.Tensor: shape=(768,), dtype=float32, numpy=
array([ 4.18643355e-01, -1.61443949e-02,  2.41889462e-01, -1.82047293e-01,
       -2.93370783e-02,  4.24753964e-01,  1.70516700e-01,  1.75103262e-01,
       -3.69384199e-01,  2.29649827e-01,  2.01773778e-01, -5.57989851e-02,
       -6.51937202e-02, -1.79486558e-01,  1.54949129e-01, -4.37164247e-01,
        7.17216671e-01,  3.49893495e-02, -5.01494016e-03, -3.46336752e-01,
       -9.99975681e-01, -2.32758805e-01, -3.65425289e-01, -5.68106584e-02,
       -3.98239195e-01,  1.46953678e-02, -2.08669260e-01, -1.08122200e-01,
        1.00351006e-01,  1.17593735e-01, -6.63065314e-02, -9.99981523e-01,
        5.58812380e-01,  7.29082465e-01,  2.12701652e-02, -2.04112723e-01,
        2.26208568e-01,  3.74048531e-01,  3.36570293e-01, -4.06370938e-01,
       -1.42808035e-01,  1.63993821e-01, -1.48139045e-01,  1.87827229e-01,
       -1.21155873e-01, -1.56604990e-01, -1.45861536e-01,  1.78934470e-01,
       -1.78421602e-01,  3.69490199e-02, -5.89816719

In [ ]:
data_list[0]

[21688483,
 'Маша и Медведь (Masha and The Bear) - Маша плюс каша (17 Серия)',
 7,
 2.6149,
 0.001738,
 0.0,
 0.0,
 1327953600]